In [13]:
#Bibliotecas externas
import pandas as pd
import pyodbc
from datetime import datetime

# String de conexão
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER=172.17.1.115;"
    f"DATABASE=dbdatastoreHouse;"
    f"UID=USR_MIS;"
    f"PWD=MIS@321"
)



#Data procurada
data_atual = datetime.today()
data_formatada = data_atual.strftime('%Y-%m-%d')

# Tentar se conectar
validaCodigoCarteira = False
indice_Filial = 0
try:
    conn = pyodbc.connect(connection_string)
    unidades = ['Marechal','Aviacao','Saverio']

    # Criar cursor para executar consultas SQL
    cursor = conn.cursor()

    # Exemplo de consulta
    query = f"""SELECT  
                    CodigoCarteira,
                    Msg
                FROM dbdatastorehouse.dbo.MapaDailyPesoToPy WITH (NOLOCK)
                WHERE DataAtualizacao = '2024-09-20'
                and Filial = {unidades[0]}
                --CONVERT(DATE, '{data_formatada}'"""

    # Executar a consulta
    cursor.execute(query)
    arrayCdCli = [];
    for row in cursor:
    # Adicionar a linha inteira ao array
        arrayCdCli.append(row)
    
    # Fechar conexão
    cursor.close()
    conn.close()
    validaCodigoCarteira = True
    print(arrayCdCli)
except Exception as e:
    print("Erro ao conectar:", e)


if (validaCodigoCarteira):
    path = f""" Z:\DailysToPy\{unidades[0]}\Daily"""
    pd.read_excel();    




[(151, 'Enviar'), (152, 'Enviar'), (155, 'Enviar'), (667, 'Enviar'), (903, 'Enviar'), (909, 'Enviar'), (913, 'Enviar'), (915, 'Enviar')]


In [34]:
import win32com.client

# Caminho do arquivo Excel
caminho_excel = r"C:\Users\t29969\Downloads\Daily_35_BV_CELULA_ESPECIALIZADA.xlsb"

# Inicia uma instância do Excel
xl = win32com.client.Dispatch('Excel.Application')

# Torna o Excel visível (opcional)
#xl.Visible = True

# Abre o arquivo Excel
workbook = xl.Workbooks.Open(Filename=caminho_excel, ReadOnly=1)

# Executa a macro
xl.Application.Run('Atualiza_Tudo')

# Salva o arquivo
workbook.Save()
workbook.Close(SaveChanges=1)

# Encerra a aplicação do Excel
xl.Application.Quit()

# Libera a variável
del xl


com_error: (-2147352567, 'Exceção.', (0, 'Microsoft Excel', "Não é possível executar a macro 'Atualiza_Tudo'. A macro poderá não estar disponível neste livro ou todas as macros poderão estar desativadas.", 'xlmain11.chm', 0, -2146827284), None)